In [65]:
from lxml import html
import requests
import unicodecsv as csv
import argparse
import json
import requests
import bs4
import pandas as pd
import time
import random

In [71]:
attrs_of_interest=[
    'list-card-addr',
    'list-card-type',
    'list-card-price',
    'list-card-details',
    'list-card-link'
]
data={}
for page in range(1,21):
    time.sleep(1)
    #url=f"https://www.zillow.com/lewisburg-pa-17837/{page}_p"
    
    searchQueryState = {
    "pagination": {"currentPage": page},
    "mapBounds": {
        "west": -77.16353057625066,
        "east": -76.40616058113348,
        "south": 40.79172057471558,
        "north": 41.214563845944205},
    "isMapVisible": True, "mapZoom": 11,
    "filterState": {"sortSelection": {"value": "globalrelevanceex"}},
    "isListVisible": True
}
    url=f"https://www.zillow.com/homes/for_sale/{page}_p/?searchQueryState={json.dumps(searchQueryState)}"
    
    print(url)
    resp = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(resp.text)    

    data_len=len(data)
    for i,x in enumerate(soup.find_all(attrs={'class': ['list-card']})):
        data[data_len+i]={}
        for y in x.find_all(attrs={'class': ['list-card-info']}):
            for z in y.find_all(attrs={'class': attrs_of_interest}):
                data[data_len+i][x.attrs['class'][0]]=x.text
                data[data_len+i][y.attrs['class'][0]]=y.text
                data[data_len+i][z.attrs['class'][0]]=z.text
                if z.attrs['class'][0]=="list-card-link":
                    homedetails_link=z.attrs['href']                    
                    data[data_len+i]["list-card-link"]=homedetails_link
                    time.sleep(random.randint(1,5))
                    homedetails_resp = requests.get(homedetails_link,headers={'User-Agent': 'Mozilla/5.0'})
                    homedetails_soup = bs4.BeautifulSoup(homedetails_resp.text)
                    for hx in homedetails_soup.find_all(attrs={'class':'ds-home-fact-list'}):
                        for hy in hx:
                            try:
                                label,value=hy.text.split(":")
                                data[data_len+i][label]=value
                            except ValueError:
                                data[data_len+i]['meta']=hy.text

https://www.zillow.com/homes/for_sale/1_p/?searchQueryState={"pagination": {"currentPage": 1}, "mapBounds": {"west": -77.16353057625066, "east": -76.40616058113348, "south": 40.79172057471558, "north": 41.214563845944205}, "isMapVisible": true, "mapZoom": 11, "filterState": {"sortSelection": {"value": "globalrelevanceex"}}, "isListVisible": true}
https://www.zillow.com/homes/for_sale/2_p/?searchQueryState={"pagination": {"currentPage": 2}, "mapBounds": {"west": -77.16353057625066, "east": -76.40616058113348, "south": 40.79172057471558, "north": 41.214563845944205}, "isMapVisible": true, "mapZoom": 11, "filterState": {"sortSelection": {"value": "globalrelevanceex"}}, "isListVisible": true}
https://www.zillow.com/homes/for_sale/3_p/?searchQueryState={"pagination": {"currentPage": 3}, "mapBounds": {"west": -77.16353057625066, "east": -76.40616058113348, "south": 40.79172057471558, "north": 41.214563845944205}, "isMapVisible": true, "mapZoom": 11, "filterState": {"sortSelection": {"value":

In [72]:
result = pd.DataFrame.from_dict(data,orient='index')

In [73]:
result.shape

(800, 17)

In [74]:
result.head(100)

,list-card,list-card-info,list-card-link,Type,Year built,Heating,Cooling,Parking,Lot,Price/sqft,list-card-addr,list-card-type,list-card-price,list-card-details,HOA,Utilities / Green Energy DetailsUtilityInternet and tv,meta
0,"10 Shady Creekside Ln, Catawissa, PA 17820Hous...","10 Shady Creekside Ln, Catawissa, PA 17820Hous...",https://www.zillow.com/homedetails/10-Shady-Cr...,Single Family,1973,No Data,No Data,1 space,No Data,$13,"10 Shady Creekside Ln, Catawissa, PA 17820",House for sale,"$25,000","4 bds1 ba1,920 sqft",NaN,NaN,NaN
1,"181 Draketown Rd, Bloomsburg, PA 17859For sale...","181 Draketown Rd, Bloomsburg, PA 17859For sale...",https://www.zillow.com/homedetails/181-Draketo...,Single Family,No Data,"Forced air, Heat pump",Central,10 spaces,8 acres,$241,"181 Draketown Rd, Bloomsburg, PA 17859",For sale by owner,"$519,000","4 bds3 ba2,150 sqft",None,NaN,NaN
2,"240 Ridge Ave, Sunbury, PA 17801House for sale...","240 Ridge Ave, Sunbury, PA 17801House for sale...",https://www.zillow.com/homedetails/240-Ridge-A...,Single Family,1920,Gas,Central,1 space,0.5 acres,$39,"240 Ridge Ave, Sunbury, PA 17801",House for sale,"$89,000","3 bds1 ba2,308 sqft",NaN,NaN,NaN
3,"10 Countryside Vlg, Selinsgrove, PA 17870For s...","10 Countryside Vlg, Selinsgrove, PA 17870For s...",https://www.zillow.com/homedetails/10-Countrys...,Mobile / Manufactured,1972,Forced air,Central,Off street,NaN,$1,"10 Countryside Vlg, Selinsgrove, PA 17870",For sale by owner,"$9,000","2 bds2 ba12,000 sqft",NaN,NaN,NaN
4,"632 State St, Danville, PA 17821For sale by ow...","632 State St, Danville, PA 17821For sale by ow...",https://www.zillow.com/homedetails/632-State-S...,Single Family,1930,No Data,No Data,1 space,No Data,$100,"632 State St, Danville, PA 17821",For sale by owner,"$139,900","3 bds2 ba1,400 sqft",None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"285 Montgomery St, Montgomery, PA 17752House f...","285 Montgomery St, Montgomery, PA 17752House f...",https://www.zillow.com/homedetails/285-Montgom...,Single Family,1950,Baseboard,None,Detached Garage,0.61 acres,$110,"285 Montgomery St, Montgomery, PA 17752",House for sale,"$192,900","3 bds2 ba1,760 sqft",NaN,NaN,NaN
96,"2110 Green Ridge Rd, Mifflinburg, PA 17844Hous...","2110 Green Ridge Rd, Mifflinburg, PA 17844Hous...",https://www.zillow.com/homedetails/2110-Green-...,Single Family,1991,Electric,Central,2 spaces,2.45 acres,$142,"2110 Green Ridge Rd, Mifflinburg, PA 17844",House for sale,"$439,000","4 bds5 ba3,088 sqft",NaN,NaN,NaN
97,"1429 Treeline Dr, Bloomsburg, PA 17815House fo...","1429 Treeline Dr, Bloomsburg, PA 17815House fo...",https://www.zillow.com/homedetails/1429-Treeli...,Single Family,1976,Radiant,Other,2 spaces,2.23 acres,$118,"1429 Treeline Dr, Bloomsburg, PA 17815",House for sale,"$249,000","4 bds3 ba2,112 sqft",NaN,NaN,NaN
98,"106 S 4th St, Lewisburg, PA 17837House for sal...","106 S 4th St, Lewisburg, PA 17837House for sal...",https://www.zillow.com/homedetails/106-S-4th-S...,Single Family,1900,Heat pump,Central,No Data,"10,454 sqft",$216,"106 S 4th St, Lewisburg, PA 17837",House for sale,"$600,000","7 bds8 ba2,780 sqft",NaN,NaN,NaN


In [75]:
result.to_csv("properties-susquehannavalley2020MAR29.csv")